In [1]:
from __future__ import division

import graphlab as gl
import pandas as pd
import pyLDAvis
import pyLDAvis.graphlab

pyLDAvis.enable_notebook()

This trial license of GraphLab Create is assigned to pbreen@uga.edu and will expire on May 23, 2016. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.


2016-04-22 19:05:43,191 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1461366341.log
/high_spd_work/sm_w2v/venv27/local/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/high_spd_work/sm_w2v/venv27/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/high_spd_work/sm_w2v/venv27/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/high_spd_work/sm_w2v/venv27/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatt

# Preprocess

In [2]:
# load and concatenate tweet timeline-documents
import json

train = False

user_timelines = dict()
with open('../data/user_timeline_tweets.json') as f_in:
    for l in f_in:
        try:
            twt = json.loads(l)
        except:
            continue
        user_id_str = twt['user']['id_str']
        if user_id_str in user_timelines:
            user_timelines[user_id_str] = user_timelines[user_id_str] + ' ' + twt['text']
        else:
            user_timelines[user_id_str] = twt['text']

In [4]:
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
import re


STOPWORDS = set(['rt', '', 'httpst', 'amp', 'https\'', 'ang',
                'ng', 'tcot', 'ako', 'sa', 'ko', 'rp', 'roc',
                'fe', 'sk', 'holi', 'asa', 'guru', 'yah', 'lista',
                 'hah' 'il', 'seo', 'casa', 'qotd', 'noh']).union(set(STOPWORDS)) \
    .union(set(stopwords.words('spanish'))) \
    .union(set(stopwords.words('portuguese'))) \
    .union(set(stopwords.words('german'))) \
    .union(set(stopwords.words('dutch')))

def clean_text(text):
    clean = re.sub(r'http.*$', '', text)
    clean = re.sub(r'[^a-zA-Z\s]', '', clean)

    clean = clean.lower()
    word_list = clean.split()
    word_list = [w for w in word_list if (w not in STOPWORDS)]

    return word_list

user_clean_timelines = dict()
for user_id_str in user_timelines.keys():
    user_clean_timelines[user_id_str] = clean_text(user_timelines[user_id_str])

/high_spd_work/sm_w2v/venv27/local/lib/python2.7/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


# Training

In [22]:
from gensim.models.ldamodel import LdaModel
from gensim.models import TfidfModel
from gensim import corpora


user_timelines_keys = []
texts = []
for k, v in user_clean_timelines.items():
    user_timelines_keys.append(k)
    texts.append(v)

# filter out users with less than 200 words in the tweets in their timelines
long_timelines = [t for t in texts if len(t) > 200]
dictionary = corpora.Dictionary(long_timelines)
dictionary.filter_extremes(no_below=10, no_above=0.3, keep_n=100000)
corpus = [dictionary.doc2bow(text) for text in long_timelines]
# tfidf = TfidfModel(corpus)
# corpus = tfidf[corpus]
corpus = corpus[0:500]
print('number of user-timelines:', len(corpus))



('number of user-timelines:', 500)


In [35]:
bows = list()
for doc in corpus:
    bow = dict()
    for word in doc:
        bow[dictionary[word[0]]] = word[1]
    bows.append(bow)


In [33]:
bows = gl.SArray(bows)

if train:
    topic_model = gl.topic_model.create(bows, num_topics=100, num_iterations=200)
    topic_model.save('../models/LDA_user_timelines')

else:
    topic_model = graphlab.load_model('../models/LDA_user_timelines')

Learning a topic model

Number of documents       500

Vocabulary size     31287

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 170       | 41.29s        | 3.94213e+06    | 0               |

| 180       | 43.63s        | 3.92917e+06    | 0               |

| 190       | 45.99s        | 3.96118e+06    | 0               |

| 200       | 48.34s        | 3.9881e+06     | 0               |

+-----------+---------------+----------------+-----------------+

# Visualization

In [34]:
pyLDAvis.graphlab.prepare(topic_model, bows)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
22     6.324365        1       1 -0.127449 -0.058993
29     3.739125        1       2 -0.184889  0.033851
0      3.263017        1       3 -0.157722  0.055120
65     3.255720        1       4 -0.127447  0.017427
23     3.190368        1       5 -0.113891 -0.108438
28     3.084990        1       6 -0.135308  0.061775
14     2.836021        1       7 -0.123893 -0.010643
70     2.551382        1       8 -0.108458  0.005830
32     2.449914        1       9 -0.149303  0.143920
45     2.254174        1      10 -0.102996 -0.101666
39     2.139356        1      11 -0.082297 -0.148265
34     2.110218        1      12 -0.156728  0.130579
8      2.091936        1      13 -0.148523  0.116000
57     2.012249        1      14 -0.086700 -0.073508
56     1.848013        1      15 -0.093060  0.000319
17     1.685601        1      16 -0.053298 -0.077121
79     1.625363        1      17 -0.099705  0.061697
96     1.603953        1      18 -0.064829 -0.115554
2      1.581048        1      19 -0.089677  0.067532
9      1.550732        1      20 -0.115013  0.107481
71     1.420342        1      21 -0.062571 -0.044465
47     1.418012        1      22 -0.061214 -0.109924
90     1.334981        1      23 -0.057299  0.013692
97     1.325333        1      24 -0.090497  0.074323
73     1.296111        1      25 -0.087827  0.114934
77     1.256527        1      26 -0.032145 -0.072181
41     1.239659        1      27 -0.066786 -0.081131
83     1.231491        1      28 -0.062569 -0.114040
18     1.205348        1      29 -0.082905  0.093200
52     1.194191        1      30 -0.054998 -0.064361
...         ...      ...     ...       ...       ...
61     0.378004        1      71  0.072859  0.000067
42     0.376622        1      72  0.065489  0.025244
44     0.367135        1      73  0.059421  0.010599
37     0.363378        1      74  0.059225  0.012768
80     0.354579        1      75  0.068409 -0.007249
55     0.342543        1      76  0.064664  0.014771
69     0.334384        1      77  0.068409  0.023842
5      0.323853        1      78  0.084959  0.008971
87     0.323360        1      79  0.074153  0.015283
31     0.316717        1      80  0.069303  0.034505
3      0.315336        1      81  0.069617  0.014599
26     0.308843        1      82  0.079346  0.027546
53     0.295086        1      83  0.072774  0.022847
91     0.287143        1      84  0.077959  0.004382
40     0.286692        1      85  0.087659  0.022032
67     0.280013        1      86  0.086027  0.023583
58     0.278132        1      87  0.080899  0.018880
49     0.274464        1      88  0.086638  0.023451
81     0.274411        1      89  0.090673  0.009791
11     0.271007        1      90  0.090421  0.016841
51     0.270934        1      91  0.085298 -0.007935
99     0.263062        1      92  0.087203 -0.008679
13     0.260603        1      93  0.089440  0.027254
93     0.240747        1      94  0.093318  0.004145
59     0.240514        1      95  0.090764  0.019878
43     0.239586        1      96  0.099181  0.024846
35     0.228359        1      97  0.098526  0.016094
95     0.214700        1      98  0.098342  0.027736
62     0.206996        1      99  0.100803  0.026866
1      0.184001        1     100  0.106007  0.023428

[100 rows x 5 columns], topic_info=       Category         Freq             Term        Total  loglift  logprob
term                                                                        
29744   Default  5824.000000     entrepreneur  5824.000000  30.0000  30.0000
26847   Default  4914.000000              kca  4914.000000  29.0000  29.0000
21597   Default  3391.000000          infosec  3391.000000  28.0000  28.0000
4326    Default  2798.000000          gadgets  2798.000000  27.0000  27.0000
9090    Default  2677.000000            pjnet  2677.000000  26.0000  26.0000
19393   Default  3041.000000         mainedcm  3041.000